<h1>Machine Learning Operations</h1>
<h2>NLP: : Emotion Recognition</h2>

<p>
  <!-- Typing SVG by DenverCoder1 - https://github.com/DenverCoder1/readme-typing-svg -->
<a href="https://git.io/typing-svg"><img src="https://readme-typing-svg.herokuapp.com?font=Fira+Code&pause=1000&random=false&width=435&lines=Natural+Language+Processing;Emotion+Recognition;By+Ahmad+Rizki" alt="Typing SVG" /></a>
</p>


### Author
- Nama: Ahmad Rizki
- Email: 18221071@std.stei.itb.ac.id
- Asal: Palu, Sulawesi Tengah

### Deskripsi
- Model Machine Learning NLP untuk mengenali emosi dari kalimat yang diberikan.
- Dataset yang digunakan adalah dataset dari [Kaggle - Emotion Datasets](https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp).
- Label  :
  - 0 : `anger`
  - 1 : `fear`
  - 2 : `joy`
  - 3 : `love`
  - 4 : `sadness`
  - 5 : `surprise`

## Import Library

Mengimport library yang digunakan untuk membangun model.

In [1]:
import tensorflow as tf
import tfx
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2
from tfx.proto import trainer_pb2
from tfx.proto import pusher_pb2 
import tensorflow_model_analysis as tfma
from tfx.dsl.components.common.resolver import Resolver 
from keras_tuner import HyperParameters
from tfx.dsl.components.common.importer import Importer
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
import absl
import pprint

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [5]:
# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

tf.get_logger().propagate = False # avoid duplicate logging
pp = pprint.PrettyPrinter()

## Set Variabel

Melakukan setup variabel yang digunakan untuk eksplorasi data dan membuat model machine learning.

In [2]:
PIPELINE_NAME = "xmrizz-pipeline"

# pipeline modules
TRANSFORM_MODULE_FILE = "modules/emotion_transform.py"
TUNER_MODULE_FILE = "modules/emotion_tuner.py"
TRAINER_MODULE_FILE = "modules/emotion_trainer.py"

# pipeline outputs
OUTPUT_BASE = "output-ipynb"
SERVING_MODEL_DIR = os.path.join(OUTPUT_BASE, 'serving_model')
PIPELINE_ROOT = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
METADATA_PATH = os.path.join(PIPELINE_ROOT, "metadata.sqlite")

Mengatur path dataset yang digunakan.

In [3]:
DATA_ROOT = './data/'

Membuat interactive context untuk pipeline

In [4]:
interactive_context = InteractiveContext(
  pipeline_root=PIPELINE_ROOT,
  metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(METADATA_PATH),
)

## Pengolahan Data

Membuat fungsi untuk menampilkan dataset.

In [6]:
def inspect_examples(component,
                     channel_name='examples',
                     split_name='train',
                     num_examples=1):
  # Get the URI of the output artifact, which is a directory
  full_split_name = 'Split-{}'.format(split_name)
  print('channel_name: {}, split_name: {} (\"{}\"), num_examples: {}\n'.format(
      channel_name, split_name, full_split_name, num_examples))
  train_uri = os.path.join(
      component.outputs[channel_name].get()[0].uri, full_split_name)

  # Get the list of files in this directory (all compressed TFRecord files)
  tfrecord_filenames = [os.path.join(train_uri, name)
                        for name in os.listdir(train_uri)]

  # Create a `TFRecordDataset` to read these files
  dataset = tf.data.TFRecordDataset(
    tfrecord_filenames, compression_type="GZIP")

  # Iterate over the records and print them
  for tfrecord in dataset.take(num_examples):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

### **Data Ingestion**

Membagi dataset yang akan digunakan untuk proses training (“train”) dan evaluasi (“eval”)

In [7]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

Membuat komponen CsvExampleGen untuk membaca dataset

In [8]:
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

interactive_context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data ./data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

Menampilkan 3 data pertama dari dataset

In [9]:
inspect_examples(example_gen, 'examples', 'train', 3)

channel_name: examples, split_name: train ("Split-train"), num_examples: 3

features {
  feature {
    key: "label"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "text"
    value {
      bytes_list {
        value: "im feeling quite sad and sorry for myself but ill snap out of it soon"
      }
    }
  }
}

features {
  feature {
    key: "label"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {
    key: "text"
    value {
      bytes_list {
        value: "i feel like a faithful servant"
      }
    }
  }
}

features {
  feature {
    key: "label"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "text"
    value {
      bytes_list {
        value: "i am just feeling cranky and blue"
      }
    }
  }
}



### **Data Validation**

#### **Summary Statistics**

Membuat komponen StatisticsGen untuk dataset

In [10]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)

interactive_context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to output-ipynb\xmrizz-pipeline\StatisticsGen\statistics\2\Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to output-ipynb\xmrizz-pipeline\StatisticsGen\statistics\2\Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

Menampilkan summary statistics dari dataset

In [11]:
interactive_context.show(statistics_gen.outputs['statistics'])

#### **Data Schema**

Membuat komponen SchemaGen untuk dataset

In [12]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])

interactive_context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to output-ipynb\xmrizz-pipeline\SchemaGen\schema\3\schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

Menampilkan data schema dari dataset

In [13]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'label',INT,required,,-
'text',BYTES,required,,-


#### **Data Anomalies**

Membuat komponen ExampleValidator untuk dataset

In [14]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

interactive_context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to output-ipynb\xmrizz-pipeline\ExampleValidator\anomalies\4\Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to output-ipynb\xmrizz-pipeline\ExampleValidator\anomalies\4\Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

Melihat data anomalies dari dataset

In [15]:
interactive_context.show(example_validator.outputs['anomalies'])

### **Data Preprocessing**

Melakukan transform pada dataset dengan module Transform

In [16]:
transform = Transform(
  examples=example_gen.outputs['examples'],
  schema=schema_gen.outputs['schema'],
  module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

INFO:absl:Generating ephemeral wheel package for 'd:\\Coding\\git-repo\\github\\Belajar\\BelajarMachineLearning\\Expert\\Machine Learning Operations\\Proyek2_MLOps_EmotionNLP\\modules\\emotion_transform.py' (including modules: ['components', 'emotion_trainer', 'emotion_transform', 'emotion_tuner']).
INFO:absl:User module package has hash fingerprint version 6ca5888ca8f3e028d2785871f0e6255d455da9365b88966707b755d419e13675.
INFO:absl:Executing: ['c:\\Users\\acer\\miniconda3\\envs\\ml-env\\python.exe', 'C:\\Users\\acer\\AppData\\Local\\Temp\\tmp83ww_x6t\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\acer\\AppData\\Local\\Temp\\tmpokr9iq5l', '--dist-dir', 'C:\\Users\\acer\\AppData\\Local\\Temp\\tmpw38ix4xq']
INFO:absl:Successfully built user code wheel distribution at 'output-ipynb\\xmrizz-pipeline\\_wheels\\tfx_user_code_Transform-0.0+6ca5888ca8f3e028d2785871f0e6255d455da9365b88966707b755d419e13675-py3-none-any.whl'; target user module is 'emotion_transform'.
INFO:ab

Instructions for updating:
Use ref() instead.


INFO:absl:Feature label has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature text has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature label has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature text has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature label has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature text has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature label has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature text has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature label has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature text has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature label has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature text has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature label has a shape dim {
  si

INFO:tensorflow:Assets written to: output-ipynb\xmrizz-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\3ed6a4ba7d7d48afa02cbb33ae652278\assets
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

Melihat hasil transform pada dataset

In [17]:
interactive_context.show(transform.outputs['post_transform_schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'label_xf',INT,required,,-
'text_xf',BYTES,required,,-


In [18]:
inspect_examples(transform, 'transformed_examples', 'train', 3)

channel_name: transformed_examples, split_name: train ("Split-train"), num_examples: 3

features {
  feature {
    key: "label_xf"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "text_xf"
    value {
      bytes_list {
        value: "im feeling quite sad sorry ill snap soon"
      }
    }
  }
}

features {
  feature {
    key: "label_xf"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {
    key: "text_xf"
    value {
      bytes_list {
        value: "feel like faithful servant"
      }
    }
  }
}

features {
  feature {
    key: "label_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "text_xf"
    value {
      bytes_list {
        value: "feeling cranky blue"
      }
    }
  }
}



## **Pemodelan**

#### **Tuning Hyperparameter**

Melakukan tuning hyperparameter untuk model dengan module Tuner

In [20]:
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(
        splits=['train'],
        num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(
        splits=['eval'],
        num_steps=1000)
)

interactive_context.run(tuner)

INFO:absl:Generating ephemeral wheel package for 'd:\\Coding\\git-repo\\github\\Belajar\\BelajarMachineLearning\\Expert\\Machine Learning Operations\\Proyek2_MLOps_EmotionNLP\\modules\\emotion_tuner.py' (including modules: ['components', 'emotion_trainer', 'emotion_transform', 'emotion_tuner']).
INFO:absl:User module package has hash fingerprint version 6ca5888ca8f3e028d2785871f0e6255d455da9365b88966707b755d419e13675.
INFO:absl:Executing: ['c:\\Users\\acer\\miniconda3\\envs\\ml-env\\python.exe', 'C:\\Users\\acer\\AppData\\Local\\Temp\\tmp82rspnzn\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\acer\\AppData\\Local\\Temp\\tmp0j08kjsi', '--dist-dir', 'C:\\Users\\acer\\AppData\\Local\\Temp\\tmp_j7eha3q']
INFO:absl:Successfully built user code wheel distribution at 'output-ipynb\\xmrizz-pipeline\\_wheels\\tfx_user_code_Tuner-0.0+6ca5888ca8f3e028d2785871f0e6255d455da9365b88966707b755d419e13675-py3-none-any.whl'; target user module is 'emotion_tuner'.
INFO:absl:Full user

Reloading Tuner from output\emotion_classification_tuning\tuner0.json


INFO:absl:Start tuning... Tuner ID: tuner0
INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'num_layers', 'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'num_units', 'default': None, 'conditions': [], 'min_value': 8, 'max_value': 64, 'step': 8, 'sampling': 'linear'}}], 'values': {'learning_rate': 0.01, 'num_layers': 1, 'num_units': 40}}
INFO:absl:Best Hyperparameters are written to output-ipynb\xmrizz-pipeline\Tuner\best_hyperparameters\6\best_hyperparameters.txt.
INFO:absl:Tuner results are written to output-ipynb\xmrizz-pipeline\Tuner\tuner_results\6\tuner_results.json.
INFO:absl:Running publisher for Tuner
INFO:absl:MetadataStore with DB connection initialized


Search space summary
Default search space size: 3
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
num_units (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 64, 'step': 8, 'sampling': 'linear'}
Results summary
Results in output\emotion_classification_tuning
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 01 summary
Hyperparameters:
learning_rate: 0.01
num_layers: 1
num_units: 40
Score: 0.857595682144165

Trial 09 summary
Hyperparameters:
learning_rate: 0.01
num_layers: 2
num_units: 32
Score: 0.8536942005157471

Trial 06 summary
Hyperparameters:
learning_rate: 0.01
num_layers: 2
num_units: 40
Score: 0.845647394657135

Trial 03 summary
Hyperparameters:
learning_rate: 0.001
num_layers: 3
num_units: 32
Score: 0.8185808062553406

Trial 02 summary
Hyperparameters:


ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

#### **Training Model**

Melakukan training model

In [21]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(
        splits=['train'],
        num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(
        splits=['eval'],
        num_steps=1000)
)

interactive_context.run(trainer)

INFO:absl:Generating ephemeral wheel package for 'd:\\Coding\\git-repo\\github\\Belajar\\BelajarMachineLearning\\Expert\\Machine Learning Operations\\Proyek2_MLOps_EmotionNLP\\modules\\emotion_trainer.py' (including modules: ['components', 'emotion_trainer', 'emotion_transform', 'emotion_tuner']).
INFO:absl:User module package has hash fingerprint version 6ca5888ca8f3e028d2785871f0e6255d455da9365b88966707b755d419e13675.
INFO:absl:Executing: ['c:\\Users\\acer\\miniconda3\\envs\\ml-env\\python.exe', 'C:\\Users\\acer\\AppData\\Local\\Temp\\tmpv9w9q3pa\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\acer\\AppData\\Local\\Temp\\tmph1i7kk82', '--dist-dir', 'C:\\Users\\acer\\AppData\\Local\\Temp\\tmpx9odsbwm']
INFO:absl:Successfully built user code wheel distribution at 'output-ipynb\\xmrizz-pipeline\\_wheels\\tfx_user_code_Trainer-0.0+6ca5888ca8f3e028d2785871f0e6255d455da9365b88966707b755d419e13675-py3-none-any.whl'; target user module is 'emotion_trainer'.
INFO:absl:Ful

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization_1 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 40)                680   

INFO:absl:Training complete. Model written to output-ipynb\xmrizz-pipeline\Trainer\model\7\Format-Serving. ModelRun written to output-ipynb\xmrizz-pipeline\Trainer\model_run\7
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## **Analisis dan Validasi Model**

### Komponen Resolver

Membuat komponen ResolverNode untuk model

In [22]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

INFO:absl:Running driver for Latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for Latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

### Komponen Evaluator

Melakukan evaluasi terhadap model yang telah dilakukan training

In [23]:
slicing_specs = [
    tfma.SlicingSpec(),
    tfma.SlicingSpec(feature_keys=[
        "gender",
        "Partner"
    ])
]

metrics_specs = [
    tfma.MetricsSpec(metrics=[
        tfma.MetricConfig(class_name='AUC'),
        tfma.MetricConfig(class_name="Precision"),
        tfma.MetricConfig(class_name="Recall"),
        tfma.MetricConfig(class_name="ExampleCount"),
        tfma.MetricConfig(
            class_name='SparseCategoricalAccuracy',
            threshold=tfma.MetricThreshold(
                value_threshold=tfma.GenericValueThreshold(
                    lower_bound={'value': 0}),
                change_threshold=tfma.GenericChangeThreshold(
                    direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                    absolute={'value': -1e-10}
                )
            )
        )
    ])
]

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='label')],
    slicing_specs=slicing_specs,
    metrics_specs=metrics_specs
)

In [25]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)

interactive_context.run(evaluator)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "AUC"\n        },\n        {\n          "class_name": "Precision"\n        },\n        {\n          "class_name": "Recall"\n        },\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "SparseCategoricalAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": -1e-10,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.0\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "label"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "gender",\n        "Partner"\n      ]\n    

INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "AUC"\n        },\n        {\n          "class_name": "Precision"\n        },\n        {\n          "class_name": "Recall"\n        },\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "SparseCategoricalAccuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": -1e-10,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.0\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "label"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "gender",\n        "Partner"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec'

INFO:absl:Evaluation complete. Results written to output-ipynb\xmrizz-pipeline\Evaluator\evaluation\9.
INFO:absl:Checking validation results.


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


INFO:absl:Blessing result True written to output-ipynb\xmrizz-pipeline\Evaluator\blessing\9.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

Memvisualisasikan hasil evaluasi model

In [26]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'accuracy': …

## **Deployment**

In [27]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR
        )
    )
)

interactive_context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1706740845
INFO:absl:Model written to serving path output-ipynb\serving_model\1706740845.
INFO:absl:Model pushed to output-ipynb\xmrizz-pipeline\Pusher\pushed_model\10.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

Memastikan model telah diexport

In [28]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

('serving_default',
 <ConcreteFunction signature_wrapper(*, examples) at 0x2E8620D8670>)
